In [1]:
# LocalSQLConnection.py script

import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
# Create universal variables for the database and connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)

2.6.0


In [3]:
# Define an execute function that can be applied to all queries in this exercise

def execute_query(query):
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    
    # Print table column names
    colnames = list(map(lambda x: x[0], cur.description))
    print(colnames)
    
    # Print table rows
    for row in rows:
        print(row)

**Q10:** Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [4]:
query2 = """
    SELECT sub.name, (SUM(sub.cost) - sub.monthlymaintenance - sub.initialoutlay) AS revenue
    FROM (
        SELECT f.name, f.facid, f.initialoutlay, f.monthlymaintenance,
            CASE WHEN b.memid = 0 THEN (f.guestcost * b.slots)
                ELSE (f.membercost * b.slots)
            END AS cost
        FROM Bookings AS b
        JOIN Facilities AS f
        ON b.facid = f.facid
        ) sub
    GROUP BY facid
    ORDER BY revenue DESC;
    """

execute_query(query2)

['name', 'revenue']
('Massage Room 1', 43351.600000000275)
('Squash Court', 8388.0)
('Massage Room 2', 7454.599999999995)
('Tennis Court 2', 6110)
('Tennis Court 1', 3660)
('Pool Table', -145)
('Table Tennis', -150)
('Snooker Table', -225)
('Badminton Court', -2143.5)


**Q11:** Produce a report of members and who recommended them in alphabetic surname,firstname order

In [5]:
query3 = """
    SELECT a1.surname, a1.firstname, (a2.firstname || ' ' || a2.surname) AS recommendedby
    FROM Members AS a1
    JOIN Members AS a2
    ON a1.recommendedby = a2.memid
    ORDER BY a1.surname;
    """

execute_query(query3)

['surname', 'firstname', 'recommendedby']
('Bader', 'Florence', 'Ponder Stibbons')
('Baker', 'Anne', 'Ponder Stibbons')
('Baker', 'Timothy', 'Jemima Farrell')
('Boothe', 'Tim', 'Tim Rownam')
('Butters', 'Gerald', 'Darren Smith')
('Coplin', 'Joan', 'Timothy Baker')
('Crumpet', 'Erica', 'Tracy Smith')
('Dare', 'Nancy', 'Janice Joplette')
('Genting', 'Matthew', 'Gerald Butters')
('Hunt', 'John', 'Millicent Purview')
('Jones', 'David', 'Janice Joplette')
('Jones', 'Douglas', 'David Jones')
('Joplette', 'Janice', 'Darren Smith')
('Mackenzie', 'Anna', 'Darren Smith')
('Owen', 'Charles', 'Darren Smith')
('Pinker', 'David', 'Jemima Farrell')
('Purview', 'Millicent', 'Tracy Smith')
('Rumney', 'Henrietta', 'Matthew Genting')
('Sarwin', 'Ramnaresh', 'Florence Bader')
('Smith', 'Jack', 'Darren Smith')
('Stibbons', 'Ponder', 'Burton Tracy')
('Worthington-Smyth', 'Henry', 'Tracy Smith')


**Q12:** Find the facilities with their usage by member, but not guests

In [6]:
# Query will count number of times each guest used each facility.

query4 = """
    WITH a1 AS (
        SELECT (m.firstname || ' ' || m.surname) AS member, sub.name AS facilityname
        FROM (
            SELECT b.memid, b.facid, f.name
            FROM Bookings AS b
            JOIN Facilities AS f
            ON b.facid = f.facid
            ) sub
        JOIN Members AS m
        ON sub.memid = m.memid
        WHERE sub.memid != 0
    )
    SELECT a1.member, a1.facilityname, COUNT(*) AS memberusage
    FROM a1
    GROUP BY member, facilityname
    ORDER BY member, facilityname;
    """

execute_query(query4)

['member', 'facilityname', 'memberusage']
('Anna Mackenzie', 'Badminton Court', 30)
('Anna Mackenzie', 'Massage Room 1', 1)
('Anna Mackenzie', 'Pool Table', 70)
('Anna Mackenzie', 'Snooker Table', 7)
('Anna Mackenzie', 'Squash Court', 2)
('Anna Mackenzie', 'Table Tennis', 16)
('Anne Baker', 'Badminton Court', 10)
('Anne Baker', 'Massage Room 1', 3)
('Anne Baker', 'Massage Room 2', 2)
('Anne Baker', 'Pool Table', 12)
('Anne Baker', 'Squash Court', 49)
('Anne Baker', 'Table Tennis', 1)
('Anne Baker', 'Tennis Court 1', 6)
('Anne Baker', 'Tennis Court 2', 35)
('Burton Tracy', 'Badminton Court', 2)
('Burton Tracy', 'Massage Room 1', 31)
('Burton Tracy', 'Pool Table', 30)
('Burton Tracy', 'Snooker Table', 20)
('Burton Tracy', 'Squash Court', 35)
('Burton Tracy', 'Table Tennis', 24)
('Burton Tracy', 'Tennis Court 1', 31)
('Burton Tracy', 'Tennis Court 2', 3)
('Charles Owen', 'Badminton Court', 6)
('Charles Owen', 'Massage Room 1', 11)
('Charles Owen', 'Massage Room 2', 2)
('Charles Owen', 'Po

**Q13:** Find the facilities usage by month, but not guests

In [7]:
# Query will extract month from the booking starttime, then count number of times each facility was used in each month.

query5 = """
        WITH a1 AS (
        SELECT strftime('%m', sub.starttime) AS month, sub.name AS facilityname
        FROM (
            SELECT b.memid, b.facid, f.name, b.starttime
            FROM Bookings AS b
            JOIN Facilities AS f
            ON b.facid = f.facid
            ) sub
        JOIN Members AS m
        ON sub.memid = m.memid
        WHERE sub.memid != 0
        )
    SELECT a1.month, a1.facilityname, COUNT(*) AS monthusage
    FROM a1
    GROUP BY month, facilityname
    ORDER BY month, facilityname;
    """

execute_query(query5)

['month', 'facilityname', 'monthusage']
('07', 'Badminton Court', 51)
('07', 'Massage Room 1', 77)
('07', 'Massage Room 2', 4)
('07', 'Pool Table', 103)
('07', 'Snooker Table', 68)
('07', 'Squash Court', 23)
('07', 'Table Tennis', 48)
('07', 'Tennis Court 1', 65)
('07', 'Tennis Court 2', 41)
('08', 'Badminton Court', 132)
('08', 'Massage Room 1', 153)
('08', 'Massage Room 2', 9)
('08', 'Pool Table', 272)
('08', 'Snooker Table', 154)
('08', 'Squash Court', 85)
('08', 'Table Tennis', 143)
('08', 'Tennis Court 1', 111)
('08', 'Tennis Court 2', 109)
('09', 'Badminton Court', 161)
('09', 'Massage Room 1', 191)
('09', 'Massage Room 2', 14)
('09', 'Pool Table', 408)
('09', 'Snooker Table', 199)
('09', 'Squash Court', 87)
('09', 'Table Tennis', 194)
('09', 'Tennis Court 1', 132)
('09', 'Tennis Court 2', 126)
